In [1]:
#include "inc/loadFISTlibs.h"

###############################################################################
#                                                                             #
# This is Thermal-FIST version 1.2.2                                          #
#                                                                             #
# Copyright (c) 2019 Volodymyr Vovchenko <vovchenko@fias.uni-frankfurt.de>    #
#                                                                             #
# Distributed under the GNU General Public License 3.0 (GPLv3 or later)       #
#                                                                             #
# Please cite when using this code:                                           #
# V. Vovchenko, H. Stoecker, arXiv:1901.05249 [nucl-th]                       #
#                                                                             #
# The latest version is available at https://github.com/vlvovch/Thermal-FIST  #
#                                       

In [2]:
using namespace std;

In [3]:
using namespace thermalfist;

In [4]:
bool useWidth  = true;
bool useQStats = true;

double Tch = 0.155;

In [5]:
// Particle list
//ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list-withnuclei.dat");
ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list.dat");

Fill the list of the considered hadron yield ratios

In [6]:
vector<int> pdgs1, pdgs2;
vector<string> names1, names2;

names1.push_back("K");
names2.push_back("pi");
pdgs1.push_back(321);
pdgs2.push_back(211);

names1.push_back("Xi");
names2.push_back("pi");
pdgs1.push_back(3312);
pdgs2.push_back(211);

names1.push_back("phi");
names2.push_back("pi");
pdgs1.push_back(333);
pdgs2.push_back(211);

names1.push_back("p");
names2.push_back("pi");
pdgs1.push_back(2212);
pdgs2.push_back(211);

names1.push_back("Om");
names2.push_back("pi");
pdgs1.push_back(3334);
pdgs2.push_back(211);

names1.push_back("La");
names2.push_back("pi");
pdgs1.push_back(3122);
pdgs2.push_back(211);

names1.push_back("K*");
names2.push_back("pi");
pdgs1.push_back(323);
pdgs2.push_back(211);

Compute the limiting GCE yield ratios values

In [7]:
vector<double> ratiosGCE;
{
    ThermalModelIdeal modelgce(&parts);

    if (useWidth)
        modelgce.SetUseWidth(ThermalParticle::eBW);
    else
        modelgce.SetUseWidth(ThermalParticle::ZeroWidth);
    modelgce.SetStatistics(useQStats);
    // Include quantum statistics for mesons only
    if (useQStats){
        for (int i = 0; i < modelgce.TPS()->Particles().size(); ++i) {
            ThermalParticle &part = modelgce.TPS()->Particle(i);
            if (part.BaryonCharge() != 0)
                part.UseStatistics(false);
        }
    }

    modelgce.SetTemperature(Tch);
    modelgce.SetBaryonChemicalPotential(0.);
    modelgce.SetElectricChemicalPotential(0.);
    modelgce.SetStrangenessChemicalPotential(0.);

    modelgce.FillChemicalPotentials();

    modelgce.CalculateDensities();

    for (int i = 0; i < names1.size(); ++i) {
        ratiosGCE.push_back(modelgce.GetDensity(pdgs1[i], 1) / modelgce.GetDensity(pdgs2[i], 1));
    }
}

Prepare the output (on the screen and to a file)

In [8]:
vector<std::ostream*> outs;
outs.push_back(&std::cout);

outs.push_back(NULL);

Routine performing the (logarithmic scale) scan over Vc and dNpi/dy.

Parameter k determines Vc = k dV/dy

In [9]:
void PerformScan(ThermalModelBase *model, 
                   double k,
                   const string& filename)
{
    ofstream fout(filename);
    outs[1] = &fout;
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "dNpipm/dy";
        *out << setw(15) << "Vc[fm3]";

        for (int i = 0; i < names1.size(); ++i) {
            *out << setw(15) << "(" + (names1[i] + "/" + names2[i]) + ")toHML";
        }

        *out << endl;
    }
    
    double Vmin = 1. * k;
    double Vmax = 15000. * k;
    int iters = 30;
    double logVmin = log10(Vmin), logVmax = log10(Vmax);
    double dlogV = (logVmax - logVmin) / (iters - 1);
    
    for (double logV = logVmin; logV <= logVmax + 1.e-3; logV += dlogV) {
        double V = pow(10., logV);

        model->SetVolume(V);
        model->SetCanonicalVolume(V);
        model->CalculateDensities();

        for(std::ostream* out : outs) {
            *out << setw(15) << 2. * model->GetDensity(211, 1) * V / k; // Charged pions dN/dy
            *out << setw(15) << V; // Vc

            for (int i = 0; i < names1.size(); ++i) {
                *out << setw(15) << model->GetDensity(pdgs1[i], 1) / model->GetDensity(pdgs2[i], 1) / ratiosGCE[i];
            }

            *out << endl;
        }
    }
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "HML";
        *out << setw(15) << "HML";

        for (int i = 0; i < ratiosGCE.size(); ++i) {
            *out << setw(15) << ratiosGCE[i];
        }

        *out << endl;
    }
    
    fout.close();
}

Perform the vanilla CSM scans

In [10]:
ThermalModelCanonical modelce(&parts);

if (useWidth)
    modelce.SetUseWidth(ThermalParticle::eBW);
else
    modelce.SetUseWidth(ThermalParticle::ZeroWidth);

modelce.SetStatistics(useQStats);
// Include quantum statistics for mesons only
if (useQStats){
    for (int i = 0; i < modelce.TPS()->Particles().size(); ++i) {
        ThermalParticle &part = modelce.TPS()->Particle(i);
        if (part.BaryonCharge() != 0)
            part.UseStatistics(false);
    }
}

modelce.SetTemperature(Tch);
modelce.SetBaryonChemicalPotential(0.);
modelce.SetElectricChemicalPotential(0.);
modelce.SetStrangenessChemicalPotential(0.);

modelce.SetBaryonCharge(0);
modelce.SetElectricCharge(0);
modelce.SetStrangeness(0);

modelce.FillChemicalPotentials();

In [11]:
PerformScan(&modelce, 1., "out/1906.03145.3_VanillaCSM.dy1.dat");

      dNpipm/dy        Vc[fm3]    (K/pi)toHML   (Xi/pi)toHML  (phi/pi)toHML    (p/pi)toHML   (Om/pi)toHML   (La/pi)toHML   (K*/pi)toHML
      0.0768496              1       0.343883     0.00291111        3.19697      0.0155662    0.000459082      0.0104024       0.128673
       0.111546        1.39317       0.354193     0.00418306        3.06855      0.0216265    0.000680352      0.0146233       0.149968
       0.164161        1.94091       0.368626     0.00608585        2.90487      0.0300473     0.00103266      0.0206458       0.178534
       0.245806        2.70402       0.388846     0.00900754        2.70283      0.0417816     0.00161876      0.0293372       0.216438
       0.375664        3.76715       0.417156      0.0136452        2.46393      0.0582317     0.00264668      0.0420808       0.266096
       0.586957        5.24826       0.456628      0.0213026        2.19708      0.0815298     0.00456028      0.0611505       0.330182
       0.935537        7.31171       0.510947   

In [12]:
PerformScan(&modelce, 3., "out/1906.03145.3_VanillaCSM.dy3.dat");

      dNpipm/dy        Vc[fm3]    (K/pi)toHML   (Xi/pi)toHML  (phi/pi)toHML    (p/pi)toHML   (Om/pi)toHML   (La/pi)toHML   (K*/pi)toHML
      0.0933613              3       0.396715      0.0102303        2.63171      0.0463451     0.00187883      0.0328086       0.230608
       0.143668         4.1795       0.428155      0.0156327        2.38268      0.0646662     0.00311945      0.0472332       0.284491
       0.225991        5.82274       0.471869      0.0246704        2.11038      0.0907199     0.00547394      0.0689698       0.353653
       0.361925        8.11205       0.531583      0.0404109        1.83601       0.128304      0.0102148       0.102429       0.440489
       0.583287        11.3014       0.609817      0.0685933        1.58734       0.183067       0.020203       0.154481       0.545478
       0.928507        15.7448       0.703856       0.118804        1.38944       0.262074      0.0415087       0.233959       0.663398
        1.43606        21.9351       0.800704   

In [13]:
PerformScan(&modelce, 6., "out/1906.03145.3_VanillaCSM.dy6.dat");

      dNpipm/dy        Vc[fm3]    (K/pi)toHML   (Xi/pi)toHML  (phi/pi)toHML    (p/pi)toHML   (Om/pi)toHML   (La/pi)toHML   (K*/pi)toHML
       0.117839              6       0.476557      0.0257559        2.08527      0.0935745     0.00577681      0.0714302       0.360751
       0.188941          8.359       0.537883      0.0423343        1.81203       0.132451      0.0108404       0.106249        0.44926
       0.304414        11.6455       0.617782      0.0720539        1.56706       0.189108      0.0215388       0.160406       0.555759
       0.483579        16.2241       0.712785       0.124851        1.37454       0.270635      0.0443289       0.242776       0.674257
       0.745693        22.6029        0.80889       0.212498        1.24206       0.381501       0.090481       0.358896       0.788376
        1.11452        31.4896        0.88566       0.338977        1.15793       0.515818       0.172907       0.501228       0.876404
        1.62791        43.8703       0.933071   